In [24]:
import torch
from torch import nn
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
class GraphConv(nn.Module):
    def __init__(self, units, features_dim, edges, dropout_rate):
        super(GraphConv, self).__init__()
        self.units = units
        self.features_dim = features_dim
        self.edges = edges
        self.dropout_rate = dropout_rate

        self.linear1 = nn.ModuleList([
            nn.Linear(self.features_dim, self.units) for _ in range(self.edges - 1)
        ])
        self.linear2 = nn.Linear(self.features_dim, self.units)
        self.tanh = nn.Tanh()
        self.dropout = nn.Dropout(self.dropout_rate)

    def forward(self, inputs):
        length = len(inputs)
        adjacency, features = inputs[0], inputs[1]
        if length > 2:
            hidden = inputs[2]
            annotations = torch.cat((hidden, features), dim=-1)
        else:
            annotations = features
        res = torch.stack([linear(annotations) for linear in self.linear1], dim=1)
        matrix = torch.matmul(adjacency[:, :-1, :, :], res)
        total = torch.sum(matrix, dim=1) + self.linear2(annotations)
        return adjacency, features, self.dropout(self.tanh(total))


In [21]:
from utils import MyDataLoader

dataloader = MyDataLoader(filepath='../../tox21.csv')
df = dataloader.load()
adj, feat, atom_mapping, bond_mapping = dataloader.preprocess(df, 10, 'smiles')
BATCH_SIZE = 32
EPOCHS = 200
DROPOUT_RATE = 0.2
LATENT_DIM = 64
BOND_DIM = adj.shape[1]
NUM_ATOMS = adj.shape[-1]
ATOM_DIM = feat.shape[-1]

# generator = GraphGenerator(
#     BATCH_SIZE, DROPOUT_RATE, LATENT_DIM, [128, 256, 512], BOND_DIM, NUM_ATOMS, ATOM_DIM
# ).to(device)

# discriminator = Discriminator(
#     [(128, 128), 128], [128, 1], ATOM_DIM, BOND_DIM, DROPOUT_RATE
# ).to(device)

adj_tensor, feat_tensor = dataloader.train_batch(adj, feat, BATCH_SIZE)

[17:54:53] WARNING: not removing hydrogen atom without neighbors
[17:54:54] WARNING: not removing hydrogen atom without neighbors


In [25]:
gconv = GraphConv(units=100, features_dim=ATOM_DIM, edges=BOND_DIM, dropout_rate=0.2).to(device)
hidden = [adj_tensor[0].to(device), feat_tensor[0].to(device)]
gconv(hidden)

(tensor([[[[0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.],
           ...,
           [0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.]],
 
          [[0., 1., 0.,  ..., 0., 0., 0.],
           [1., 0., 1.,  ..., 0., 0., 0.],
           [0., 1., 0.,  ..., 0., 0., 0.],
           ...,
           [0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.]],
 
          [[0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.],
           ...,
           [0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.]],
 
          [[0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.],
           ...,
           [0., 0., 0., 